# 自然语言生成与神经机器翻译 (Natural Language Generation and Neural Machine Translation)
介绍自然语言生成（NLG）的基本概念、相关模型，特别是循环神经网络语言模型（RNN-LM）及其在机器翻译（MT）中的应用，包括训练方法、解码策略、评估指标以及神经机器翻译（NMT）的发展历程和核心技术。

## 1. 生成模型 (Generative Models)

P物理学家理查德·费曼（Richard Feynman）的名言：“我无法创造的，我就不理解。”（What I cannot create, I do not understand.）这句话深刻地阐述了生成模型的理念：要真正理解一个现象，就必须能够模拟和生成它。在人工智能领域，生成模型旨在学习数据的内在分布，从而能够生成新的、与真实数据相似的样本。

## 2. 自然语言生成 (Natural Language Generation, NLG)

自然语言生成是指任何生成（或“编写”）新文本的场景或任务。它是一个广阔的领域，包括多种子任务：

*   **机器翻译 (Machine Translation)**：将一种语言的文本翻译成另一种语言。
*   **(抽象式) 文本摘要 ((Abstractive) Summarization)**：理解文本内容后，生成一个更短、但保留关键信息的新文本，而非简单地从原文中抽取句子。
*   **对话系统 (Dialogue Systems)**：包括闲聊（chit-chat）和任务型对话（task-based），生成连贯且有意义的回应。
*   **创意写作 (Creative Writing)**：如故事创作、诗歌生成等。
*   **自由形式问答 (Freeform Question Answering)**：生成的答案，而非从文本或知识库中直接提取。
*   **图像描述 (Image Captioning)**：根据图像内容生成相应的文字描述。

In [1]:
# 简单的文本生成函数示意
def generate_text(task_type, input_data=None):
    """
    一个象征性的文本生成函数。
    在实际NLG系统中，这会涉及复杂的模型推理。
    """
    if task_type == "机器翻译":
        if input_data:
            return f"源语言：'{input_data}' -> 目标语言：'这是一段翻译后的文本。'"
        else:
            return "请输入待翻译的文本。"
    elif task_type == "文本摘要":
        if input_data:
            return f"原文：'{input_data[:50]}...' -> 摘要：'这是原文的精简摘要。'"
        else:
            return "请输入待摘要的文本。"
    elif task_type == "对话":
        if input_data:
            responses = {
                "你好": "你好！有什么我可以帮你的吗？",
                "天气怎么样": "今天天气很好。",
                "再见": "再见！祝您愉快！"
            }
            return f"用户：'{input_data}' -> 助手：'{responses.get(input_data, '抱歉，我不明白。')}'"
        else:
            return "请开始对话。"
    else:
        return "未知生成任务类型。"

print(generate_text("机器翻译", "Hello, how are you?"))
print(generate_text("文本摘要", "自然语言处理是人工智能领域的一个重要分支，它研究如何让计算机理解和生成人类语言，包括文本摘要、机器翻译、情感分析等多个子领域。"))
print(generate_text("对话", "你好"))
print(generate_text("对话", "今天天气怎么样"))

源语言：'Hello, how are you?' -> 目标语言：'这是一段翻译后的文本。'
原文：'自然语言处理是人工智能领域的一个重要分支，它研究如何让计算机理解和生成人类语言，包括文本摘要、机器翻...' -> 摘要：'这是原文的精简摘要。'
用户：'你好' -> 助手：'你好！有什么我可以帮你的吗？'
用户：'今天天气怎么样' -> 助手：'抱歉，我不明白。'


## 3. 循环神经网络语言模型 (RNN-LM)

**语言建模 (Language Modeling)** 的任务是预测给定目前为止的词语序列的下一个词。
其概率表示为：$P(y_t | y_1, ..., y_{t-1})$

一个生成这种概率分布的系统称为**语言模型 (Language Model)**。如果这个系统是一个循环神经网络 (RNN)，则称为 **RNN-LM**。

**条件语言建模 (Conditional Language Modeling)** 的任务是预测给定目前为止的词语序列，以及一些其他输入 `x` 的下一个词。
其概率表示为：$P(y_t | y_1, ..., y_{t-1}, x)$

条件语言建模的例子包括：
*   **机器翻译**：`x` = 源句子，`y` = 目标句子。
*   **文本摘要**：`x` = 输入文本，`y` = 摘要文本。
*   **对话系统**：`x` = 对话历史，`y` = 下一个对话回合。

In [2]:
import random
from collections import defaultdict

class SimpleLanguageModel:
    def __init__(self):
        self.transitions = defaultdict(lambda: defaultdict(int))
        self.context_counts = defaultdict(int)

    def train(self, text_corpus):
        words = text_corpus.split()
        for i in range(len(words) - 1):
            current_word = words[i]
            next_word = words[i+1]
            self.transitions[current_word][next_word] += 1
            self.context_counts[current_word] += 1

    def predict_next_word(self, current_word):
        if current_word not in self.transitions:
            return None # 无法预测
        
        possible_next_words = self.transitions[current_word]
        total_count = self.context_counts[current_word]
        
        if total_count == 0:
            return None
            
        # 计算概率分布
        probabilities = {
            word: count / total_count 
            for word, count in possible_next_words.items()
        }
        
        # 根据概率随机选择下一个词 (这里简化为选择概率最高的)
        # 实际更复杂，会用到softmax和采样
        if not probabilities:
            return None
        
        return max(probabilities, key=probabilities.get)
    
    def generate_sentence(self, start_word, max_length=10):
        sentence = [start_word]
        current_word = start_word
        for _ in range(max_length - 1):
            next_word = self.predict_next_word(current_word)
            if next_word is None:
                break
            sentence.append(next_word)
            current_word = next_word
        return " ".join(sentence)

# 训练一个简单的语言模型
corpus = "I love this movie. I hate this movie. This movie is great."
lm = SimpleLanguageModel()
lm.train(corpus)

print(f"预测 'love' 后面的词: {lm.predict_next_word('love')}")
print(f"生成句子从 'I' 开始: {lm.generate_sentence('I')}")
print(f"生成句子从 'This' 开始: {lm.generate_sentence('This')}")

预测 'love' 后面的词: this
生成句子从 'I' 开始: I love this movie. I love this movie. I love
生成句子从 'This' 开始: This movie is great.


### 3.1 编码器-解码器架构 (Encoder-Decoder Architecture)

*   **编码器 (Encoder)**：读取源句子（例如，英文），将其编码成一个**上下文向量 (context vector)**，这个向量包含了源句子的所有信息。
*   **解码器 (Decoder)**：接收编码器生成的上下文向量，并逐步生成目标句子（例如，中文）。解码器本身也是一个语言模型，它在生成每个词时，都会考虑之前生成的词和编码器提供的上下文信息。

**隐藏状态传递方式** (How to Pass Hidden State?)：

1.  **初始化解码器**：用编码器最终的隐藏状态来初始化解码器的隐藏状态（Sutskever et al. 2014）。
2.  **转换**：对编码器状态进行转换（维度可能不同）后，再传递给解码器。
3.  **每步输入**：将编码器状态作为解码器每一步的输入（Kalchbrenner & Blunsom 2013）。

### 3.2 训练 (Training)

训练条件RNN-LM（如机器翻译模型）通常使用**教师强制 (Teacher Forcing)** 方法。
*   在训练过程中，解码器在生成当前词时，其输入不是前一步预测的词，而是真实（参考）目标句子中的前一个词。
*   这种方法能确保解码器在训练时始终获得正确的历史信息，从而加速训练并提高稳定性。

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

# 简化版Encoder
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.rnn = nn.LSTM(input_dim, hidden_dim)

    def forward(self, src):
        # src: (seq_len, batch_size, input_dim)
        outputs, (hidden, cell) = self.rnn(src)
        return hidden, cell

# 简化版Decoder
class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim):
        super().__init__()
        self.rnn = nn.LSTM(output_dim, hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, output_dim)

    def forward(self, input, hidden, cell):
        # input: (1, batch_size, output_dim)
        output, (hidden, cell) = self.rnn(input, (hidden, cell))
        # output: (1, batch_size, hidden_dim)
        prediction = self.fc_out(output.squeeze(0))
        # prediction: (batch_size, output_dim)
        return prediction, hidden, cell

# 模拟一个端到端的Seq2Seq模型
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src: (src_seq_len, batch_size, src_dim)
        # trg: (trg_seq_len, batch_size, trg_dim)
        
        batch_size = trg.shape[1]
        trg_seq_len = trg.shape[0]
        trg_dim = trg.shape[2]
        
        # 存储解码器输出
        outputs = torch.zeros(trg_seq_len, batch_size, trg_dim).to(self.device)
        
        # 编码器处理源句子
        encoder_hidden, encoder_cell = self.encoder(src)
        
        # 解码器的第一个输入是特殊<SOS> token (这里简化为全零向量)
        input = torch.zeros(1, batch_size, trg_dim).to(self.device)
        
        hidden = encoder_hidden
        cell = encoder_cell

        for t in range(trg_seq_len):
            prediction, hidden, cell = self.decoder(input, hidden, cell)
            
            outputs[t] = prediction # 存储预测结果

            # Teacher Forcing: 有一定概率使用真实目标词作为下一个输入
            teacher_force = random.random() < teacher_forcing_ratio
            
            # 真实目标词的嵌入
            ground_truth_input = trg[t].unsqueeze(0) 
            
            # 选择下一个输入
            input = ground_truth_input if teacher_force else prediction.unsqueeze(0) # 预测结果需要unsqueeze来匹配输入维度
            
        return outputs

# 模型参数
INPUT_DIM = 10  # 模拟词嵌入维度
OUTPUT_DIM = 10 # 模拟词嵌入维度 (这里简化为一样)
HIDDEN_DIM = 20

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

encoder = Encoder(INPUT_DIM, HIDDEN_DIM).to(device)
decoder = Decoder(OUTPUT_DIM, HIDDEN_DIM).to(device)
model = Seq2Seq(encoder, decoder, device).to(device)

# 模拟数据
SRC_SEQ_LEN = 5
TRG_SEQ_LEN = 7
BATCH_SIZE = 4

src_data = torch.randn(SRC_SEQ_LEN, BATCH_SIZE, INPUT_DIM).to(device) # 源句子序列
trg_data = torch.randn(TRG_SEQ_LEN, BATCH_SIZE, OUTPUT_DIM).to(device) # 目标句子序列 (ground truth)

# 模拟训练步骤
optimizer = optim.Adam(model.parameters())
criterion = nn.MSELoss() # 简化为MSE Loss

model.train()
output = model(src_data, trg_data)
loss = criterion(output, trg_data) # 计算损失
loss.backward() # 反向传播
optimizer.step() # 更新参数

print(f"模拟训练损失: {loss.item():.4f}")
print("模型结构：")
print(model)

模拟训练损失: 0.9438
模型结构：
Seq2Seq(
  (encoder): Encoder(
    (rnn): LSTM(10, 20)
  )
  (decoder): Decoder(
    (rnn): LSTM(10, 20)
    (fc_out): Linear(in_features=20, out_features=10, bias=True)
  )
)


## 4. 生成策略 (The Generation Problem)

给定一个 $P(Y|X)$ 模型，我们如何使用它来生成句子 $Y$？主要有两种方法：

1.  **采样 (Sampling)**：根据概率分布生成一个随机句子。
    *   **祖先采样 (Ancestral Sampling)**：一个词一个词地随机生成。`y_j ~ P(y_j | X, y_1, ..., y_{j-1})`
        *   **纯采样 (Pure Sampling)**：在每一步 $t$，从 $P_t$ 概率分布中随机采样下一个词。
        *   **Top-n 采样 (Top-n Sampling)**：在每一步 $t$，只从 $P_t$ 中概率最高的 Top-n 个词中进行采样。
            *   n=1 等同于贪婪搜索。
            *   n=V (词汇表大小) 等同于纯采样。
            *   增加 n 获得更多样/有风险的输出。
            *   减少 n 获得更通用/安全的输出。
    *   **优点**：生成多样性，适合开放式/创意生成（诗歌、故事）。
    *   **缺点**：每次生成可能不同，难以控制质量。

2.  **贪婪搜索 (Greedy Search)**：尝试生成概率最高的句子。
    *   **策略**：在每一步，选择概率最高的词（即 `argmax`）。将该词作为下一个输入，并重复此过程，直到生成 `<END>` 标记或达到最大长度。
    *   **优点**：简单。
    *   **缺点**：缺乏回溯机制，可能错过全局最优解。因为它只考虑当前步的最优选择，而不考虑对未来步骤的影响，容易陷入局部最优，导致生成质量较差。例如，“I arrived at the **bank** of the river.” 如果“bank”在当前步概率最高，但后面的词是“money”，则会生成“I arrived at the money”，这显然是错的。

3.  **束搜索 (Beam Search)**：一种启发式搜索算法，旨在找到高概率序列（不一定是全局最优）。
    *   **核心思想**：在解码器的每一步，跟踪 $k$ 个最可能的**部分序列 (partial sequences)**，这些序列被称为**假设 (hypotheses)**。
    *   $k$ 是**束大小 (beam size)**。
    *   达到停止条件后，选择 $k$ 个假设中概率最高的序列（通常会对长度进行调整）。
    *   **优点**：
        *   比贪婪搜索质量更好。
        *   增加了搜索空间，能找到比贪婪搜索更好的序列。
    *   **缺点**：
        *   计算成本更高（与 $k$ 成正比）。
        *   如果 $k$ 过大，可能会产生高概率但**不适合 (unsuitable)** 的输出（例如，在NMT中，过大的 $k$ 可能导致翻译过短，甚至降低BLEU分数）。
        *   在开放式任务（如闲聊）中，大的 $k$ 可能导致输出更**通用 (generic)**。

### 4.1 Softmax 温度 (Softmax Temperature)

Softmax 温度 $τ$ 是一个超参数，用于调整 Softmax 函数的输出分布，从而影响采样的随机性。
$P_t(w) = \frac{\exp(s_w/\tau)}{\sum_{w' \in V} \exp(s_{w'}/\tau)}$

*   **提高温度 $τ$**：$P_t$ 变得更**均匀 (uniform)**。这意味着概率会分散到更多词上，导致更多样化（更“有风险”）的输出。
*   **降低温度 $τ$**：$P_t$ 变得更**尖锐 (spiky)**。这意味着概率会集中在少数高概率词上，导致更少多样化（更“安全”）的输出。
*   **注意**：Softmax 温度不是一种解码算法，而是一种可以在测试时与任何解码算法（如束搜索或采样）结合使用的技术。

In [4]:
import numpy as np
import torch
import torch.nn.functional as F

# 模拟模型输出的log_probs (例如来自解码器最后一层的输出)
def simulate_model_output_logits(vocab_size, num_steps):
    """模拟一个序列中每一步的词汇表logit输出"""
    return [np.random.rand(vocab_size) * 5 - 2.5 for _ in range(num_steps)] # 随机logit

def softmax_with_temperature(logits, temperature=1.0):
    """计算带温度的softmax概率"""
    logits_tensor = torch.tensor(logits, dtype=torch.float32)
    return F.softmax(logits_tensor / temperature, dim=-1).numpy()

# 词汇表映射
vocab = ["I", "love", "hate", "this", "movie", "book", "is", "great", "<END>"]
vocab_size = len(vocab)
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for i, word in enumerate(vocab)}

print("--- 模拟 Softmax 温度 ---")
logits_step1 = simulate_model_output_logits(vocab_size, 1)[0]
print(f"原始Logits (部分): {logits_step1[:5]}...")

probs_t1 = softmax_with_temperature(logits_step1, temperature=1.0)
print(f"温度 1.0 (默认) 概率 (部分): {probs_t1[:5]}...")
# 找到概率最高的词
print(f"温度 1.0 下最可能的词: {idx_to_word[np.argmax(probs_t1)]}")


probs_t0_1 = softmax_with_temperature(logits_step1, temperature=0.1) # 降低温度，更尖锐
print(f"温度 0.1 (低) 概率 (部分): {probs_t0_1[:5]}...")
print(f"温度 0.1 下最可能的词: {idx_to_word[np.argmax(probs_t0_1)]}")


probs_t2 = softmax_with_temperature(logits_step1, temperature=2.0) # 提高温度，更均匀
print(f"温度 2.0 (高) 概率 (部分): {probs_t2[:5]}...")
print(f"温度 2.0 下最可能的词: {idx_to_word[np.argmax(probs_t2)]}")


print("\n--- 解码策略示例 ---")
# 模拟多步输出的logits
simulated_logits_sequence = simulate_model_output_logits(vocab_size, 3)

def greedy_decode(logits_sequence):
    decoded_sentence = []
    for logits in logits_sequence:
        probs = softmax_with_temperature(logits, temperature=1.0) # 默认温度
        next_word_idx = np.argmax(probs)
        decoded_sentence.append(idx_to_word[next_word_idx])
        if idx_to_word[next_word_idx] == "<END>":
            break
    return " ".join(decoded_sentence)

def sample_decode(logits_sequence, temperature=1.0, top_k=None):
    decoded_sentence = []
    for logits in logits_sequence:
        probs = softmax_with_temperature(logits, temperature)
        
        if top_k is not None:
            # 找到top_k的索引和值
            top_k_indices = np.argsort(probs)[-top_k:]
            top_k_probs = probs[top_k_indices]
            # 重新归一化概率
            top_k_probs = top_k_probs / np.sum(top_k_probs)
            
            next_word_idx = np.random.choice(top_k_indices, p=top_k_probs)
        else:
            next_word_idx = np.random.choice(len(probs), p=probs)
        
        decoded_sentence.append(idx_to_word[next_word_idx])
        if idx_to_word[next_word_idx] == "<END>":
            break
    return " ".join(decoded_sentence)

print(f"贪婪解码结果: {greedy_decode(simulated_logits_sequence)}")
print(f"纯采样解码结果 (T=1.0): {sample_decode(simulated_logits_sequence, temperature=1.0)}")
print(f"Top-2采样解码结果 (T=1.0): {sample_decode(simulated_logits_sequence, temperature=1.0, top_k=2)}")
print(f"纯采样解码结果 (T=0.5): {sample_decode(simulated_logits_sequence, temperature=0.5)}")

--- 模拟 Softmax 温度 ---
原始Logits (部分): [-0.6172728   1.16680808  1.9690523   0.25522011  1.31970019]...
温度 1.0 (默认) 概率 (部分): [0.02760155 0.16434258 0.36657286 0.06604691 0.19149183]...
温度 1.0 下最可能的词: hate
温度 0.1 (低) 概率 (部分): [5.8470108e-12 3.2741512e-04 9.9816185e-01 3.5985035e-08 1.5104305e-03]...
温度 0.1 下最可能的词: hate
温度 2.0 (高) 概率 (部分): [0.06216127 0.15167995 0.22653393 0.09615666 0.16373   ]...
温度 2.0 下最可能的词: hate

--- 解码策略示例 ---
贪婪解码结果: <END>
纯采样解码结果 (T=1.0): great I hate
Top-2采样解码结果 (T=1.0): great <END>
纯采样解码结果 (T=0.5): is <END>


### 4.2 解码算法总结

*   **贪婪解码**：简单，但输出质量低。
*   **束搜索**：能搜索到高概率输出，比贪婪好。但束大小过大会导致不合适的输出（如过短、通用）。
*   **采样方法**：提供多样性和随机性，适合开放式/创意生成。Top-n 采样允许控制多样性。
*   **Softmax 温度**：控制输出多样性的另一种方式，可与任何解码算法结合使用。

### 4.3 集成 (Ensembling)

集成是指结合多个模型的预测来生成最终输出。
*   **原因**：
    *   多个模型可能产生不相关的错误，集成可以抵消这些错误。
    *   模型在不确定时更容易犯错，集成有助于平滑这种不确定性。
    *   平滑模型固有的特殊性或偏差。

## 5. 评估 (Evaluation)

评估生成模型（特别是机器翻译）的质量是一个挑战。

### 5.1 人工评估 (Human Evaluation)

*   **方法**：让人类评估员对生成的文本进行评分，通常从**充分性 (Adequacy)**（意义是否保留）和**流畅性 (Fluency)**（语法是否正确、是否自然）两个维度进行。
*   **优点**：最终目标，能真实反映质量。
*   **缺点**：耗时、昂贵、有时不一致。

### 5.2 自动评估指标

*   **BLEU (Bilingual Evaluation Understudy)**
    *   **原理**：通过比较机器翻译的输出与一个或多个参考翻译之间的 n-gram 重叠度来评估。
    *   **计算**：
        1.  计算各种 n-gram (unigram, bigram, trigram, 4-gram等) 的精度。
        2.  计算**简洁惩罚 (Brevity Penalty, BP)**，惩罚比参考翻译短的机器翻译结果。BP = `min(1, |System| / |Reference|)`。
        3.  最终 BLEU 分数是经过 BP 调整的几何平均 n-gram 精度。例如，`BLEU-N = BP * (Product_of_n-gram_precisions)^(1/N)`。
    *   **优点**：易于使用，适合衡量系统改进（在同一系统上）。
    *   **缺点**：往往与人类评估不完全一致，不适合比较差异很大的系统。

*   **METEOR (Metric for Evaluation of Translation with Explicit Ordering)**
    *   **原理**：在BLEU的基础上增加了更多技巧，如考虑同义词（paraphrases）、词序（reordering）以及功能词/内容词的区别。
    *   **优点**：通常比BLEU能更好地与人类评估相关联，特别对于高资源语言。
    *   **缺点**：需要额外资源（如同义词词典），更复杂。

*   **困惑度 (Perplexity)**
    *   **原理**：衡量语言模型对测试集（held-out set）中词语的预测能力，而不进行实际生成。困惑度越低表示模型对语言建模得越好。
    *   **计算**：$PP = (\prod_{i=1}^{N} P(w_i | w_1...w_{i-1}))^{-1/N}$
    *   **优点**：自然地解决了多参考问题。
    *   **缺点**：不考虑解码过程或实际生成输出。在有大量歧义的问题中可能适用。

In [5]:
from collections import Counter
import math

def calculate_ngram_precision(candidate, reference, n):
    candidate_ngrams = Counter(tuple(candidate[i:i+n]) for i in range(len(candidate) - n + 1))
    reference_ngrams = Counter(tuple(reference[i:i+n]) for i in range(len(reference) - n + 1))

    clipped_count = 0
    total_count = sum(candidate_ngrams.values())

    for ngram, count in candidate_ngrams.items():
        clipped_count += min(count, reference_ngrams[ngram])
    
    return clipped_count / total_count if total_count > 0 else 0

def calculate_brevity_penalty(candidate, reference):
    candidate_len = len(candidate)
    reference_len = len(reference)
    
    if candidate_len == 0:
        return 0
    if candidate_len > reference_len:
        return 1.0
    else:
        return math.exp(1 - reference_len / candidate_len)

def simple_bleu(candidate_sentence, reference_sentence, max_n=2):
    candidate_words = candidate_sentence.lower().split()
    reference_words = reference_sentence.lower().split()

    precisions = []
    for n in range(1, max_n + 1):
        precisions.append(calculate_ngram_precision(candidate_words, reference_words, n))

    bp = calculate_brevity_penalty(candidate_words, reference_words)

    # 几何平均 n-gram 精度
    geometric_mean = 1.0
    for p in precisions:
        if p == 0: # 如果某个n-gram精度为0，则整个BLEU分数为0
            return 0.0
        geometric_mean *= p
    
    # 考虑 N 值
    if len(precisions) > 0:
        geometric_mean = geometric_mean ** (1.0 / len(precisions))
    else:
        geometric_mean = 0.0 # No n-gram precisions computed

    return bp * geometric_mean

# PPT示例
reference = "Taro visited Hanako"
system_output = "the Taro visited the Hanako"

print("--- BLEU 示例 (简化版) ---")
print(f"参考: {reference}")
print(f"系统输出: {system_output}")

# 1-gram 精度
ref_1gram = Counter(reference.lower().split())
sys_1gram = Counter(system_output.lower().split())
clipped_1gram = min(sys_1gram['taro'], ref_1gram['taro']) + \
                min(sys_1gram['visited'], ref_1gram['visited']) + \
                min(sys_1gram['hanako'], ref_1gram['hanako'])
precision_1gram = clipped_1gram / len(system_output.split())
print(f"1-gram 精度: {precision_1gram:.4f} (对应PPT中的3/5)")

# 2-gram 精度
ref_2gram = Counter([("taro", "visited"), ("visited", "hanako")])
sys_2gram = Counter([("the", "taro"), ("taro", "visited"), ("visited", "the"), ("the", "hanako")])
clipped_2gram = min(sys_2gram[("taro", "visited")], ref_2gram[("taro", "visited")])
precision_2gram = clipped_2gram / (len(system_output.split()) - 1)
print(f"2-gram 精度: {precision_2gram:.4f} (对应PPT中的1/4)")


# Brevity Penalty
len_sys = len(system_output.split()) # 5
len_ref = len(reference.split())    # 3
bp = min(1, len_sys / len_ref)
print(f"简洁惩罚 (BP): {bp:.4f} (对应PPT中的min(1, 5/3) = 1.0)")

# BLEU-2 
bleu_2_val = math.sqrt(precision_1gram * precision_2gram) * bp
print(f"BLEU-2 (手动计算): {bleu_2_val:.4f} (对应PPT中的0.387)")

# 使用简单函数计算
bleu_score_custom = simple_bleu(system_output, reference, max_n=2)
print(f"BLEU-2 (函数计算): {bleu_score_custom:.4f}")

# 更真实的BLEU计算，可以使用nltk库
try:
    from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
    # 注意：nltk的sentence_bleu需要参考句子列表，即使只有一个参考
    # weights: 1-gram, 2-gram, 3-gram, 4-gram
    # smoothing_function: 为了避免0精度的问题，特别是短句子
    
    reference_tokenized = [reference.lower().split()]
    candidate_tokenized = system_output.lower().split()
    
    # BLEU-2 权重 (0.5 for 1-gram, 0.5 for 2-gram)
    weights_bleu2 = (0.5, 0.5, 0, 0) 
    
    # 使用平滑函数避免0值，确保所有n-gram都计算
    chencherry = SmoothingFunction() 
    
    nltk_bleu2 = sentence_bleu(reference_tokenized, candidate_tokenized, 
                               weights=weights_bleu2, 
                               smoothing_function=chencherry.method1)
    print(f"NLTK BLEU-2: {nltk_bleu2:.4f}")

    # NLTK默认是BLEU-4，权重 (0.25, 0.25, 0.25, 0.25)
    nltk_bleu4 = sentence_bleu(reference_tokenized, candidate_tokenized, 
                               smoothing_function=chencherry.method1)
    print(f"NLTK BLEU-4 (default): {nltk_bleu4:.4f}")

except ImportError:
    print("\nNLTK库未安装。请运行 'pip install nltk' 进行安装以获取更准确的BLEU计算。")

--- BLEU 示例 (简化版) ---
参考: Taro visited Hanako
系统输出: the Taro visited the Hanako
1-gram 精度: 0.6000 (对应PPT中的3/5)
2-gram 精度: 0.2500 (对应PPT中的1/4)
简洁惩罚 (BP): 1.0000 (对应PPT中的min(1, 5/3) = 1.0)
BLEU-2 (手动计算): 0.3873 (对应PPT中的0.387)
BLEU-2 (函数计算): 0.3873
NLTK BLEU-2: 0.3873
NLTK BLEU-4 (default): 0.1257


## 6. 神经机器翻译 (Neural Machine Translation, NMT)

NMT 是一种使用单个神经网络进行机器翻译的方法。其核心架构是**序列到序列 (sequence-to-sequence, Seq2Seq)** 模型，它通常涉及两个 RNNs（或更现代的 Transformer 模型）。

### 6.1 NMT 的发展历程 (Brief History of NMT)

*   **1950s：早期机器翻译 (Early Machine Translation)**
    *   主要是基于规则的系统，依赖双语词典进行词语映射。
*   **1990s-2010s：统计机器翻译 (Statistical Machine Translation, SMT)**
    *   核心思想：从数据中学习概率模型。目标是找到最有可能的目标句子 `y`，给定源句子 `x`：`argmax_y P(y|x)`。
    *   通过**贝叶斯规则**分解为两个独立学习的组件：`argmax_y P(x|y)P(y)`。
        *   `P(x|y)`：翻译模型（如何将目标语言译回源语言）。
        *   `P(y)`：语言模型（目标语言的流畅度）。
    *   SMT 依赖大量的并行数据（人肉翻译的源-目标句子对）。
    *   **对齐 (Alignment)** 是 SMT 的关键挑战。对齐指翻译句子对中特定词语之间的对应关系，它非常复杂，可能是一对多、多对一甚至多对多（短语级别）的。

*   **2014年至今：神经机器翻译 (NMT)**
    *   **序列到序列模型 (Seq2Seq)** (NIPS 2014)：
        *   早期 NMT 方法，直接估计 $P(Y|X)$。
        *   无需显式对齐即可学习翻译。
        *   **缺点**：所有信息必须由内部状态承载，对于长句子翻译质量会下降。
    *   **注意力模型 (Attention Models)** (2014)：
        *   解决了 Seq2Seq 对长句子的处理问题。
        *   允许解码器在生成每个词时，**关注 (attend to)** 编码器所有状态中的相关部分，从而使翻译质量不再受限于句子长度。
        *   这是 NMT 取得突破性进展的关键。
    *   **其他改进**：
        *   **双向编码器 (Bi-directional Encoder)**：编码器同时处理正向和反向序列，使每个词的表示都能总结其前后所有词的信息。
        *   **深度学习 (Deep Learning)**：堆叠多层编码器和解码器，增加模型容量。
        *   **并行化 (Parallelization)**：通过分层计算和残差连接，提高训练效率。
        *   **残差连接 (Residual Connections)**：解决深度网络中的梯度消失问题，使模型更容易训练。

NMT 已经在工业界广泛应用，如谷歌翻译、Facebook、微软翻译等都转向了NMT。

### 6.2 NMT 的优势 (Neural MT)

“乞力马扎罗的雪”的例子，新版（NMT）翻译在流畅性和准确性上明显优于旧版（SMT）。NMT 在亚洲语言（如中文、日语、韩语）上取得了显著的改进，甚至超过了过去十年所有改进的总和。在多项语言对上，NMT 都表现出超越短语基础机器翻译（PBMT）并接近人类翻译的质量。

### 6.3 挑战与扩展

尽管NMT表现出色，但训练一个高质量的模型需要大量资源：
*   **训练时间**：每个模型2-3周。
*   **训练数据**：每个模型1亿+数据。
*   **模型数量**：大规模系统可能需要大量的模型组合。

**多语言模型 (Multilingual Model)**：
*   在单个模型中训练多个语言对。令人惊讶的是，这竟然成功了！
*   通过在源语言前添加一个额外的token来指示目标语言，例如 `<2es> How are you </s>` 表示翻译成西班牙语。
*   **零样本翻译 (Zero-Shot Translation)**：
    *   如果模型训练过 A->B 和 B->C，它可能在没有直接训练 A->C 的情况下也能进行翻译。
    *   这表明模型学习到了一种**中间语言表示 (interlingua representation)**，使得不同语言的语义在嵌入空间中是语义等同的。PPT中的 t-SNE 可视化也支持了这一观点。
*   **源语言混用 (Source Language Code-Switching)**：模型甚至能处理源语言中混用不同语言的情况，并给出正确的翻译。
*   **加权目标语言选择 (Weighted Target Language Selection)**：通过对表示不同目标语言的token进行线性组合，可以控制翻译的风格或偏向。

## 7. 额外内容：Transformer 架构

主要围绕RNN/LSTM展开，但自2017年以来，**Transformer** 架构（由 Vaswani et al. 提出）已经成为 NMT 及整个 NLP 领域的事实标准。Transformer 完全放弃了循环和卷积，而是纯粹基于**自注意力机制 (Self-Attention)**。

### Transformer 的核心优势：

1.  **并行化 (Parallelization)**：自注意力机制允许模型同时处理序列中的所有词，而不是像 RNN 那样按顺序处理。这大大加快了训练速度，特别是在GPU上。
2.  **长距离依赖 (Long-Range Dependencies)**：通过自注意力，模型可以直接计算序列中任意两个词之间的关联度，无论它们距离多远。这比 RNN 更有效地捕获长距离依赖。
3.  **可解释性 (Interpretability)**：注意力权重可以被可视化，显示模型在生成某个词时，源句子中的哪些部分是“重要”的，这提供了一定程度的可解释性。
4.  **无循环结构 (Absence of Recurrence)**：简化了模型设计，避免了 RNN 中梯度消失/爆炸的问题。

**Transformer 结构示意：**
*   **编码器 (Encoder)**：由多个相同的层堆叠而成。每层包含一个多头自注意力层（Multi-Head Self-Attention）和一个前馈网络（Feed-Forward Network）。
*   **解码器 (Decoder)**：也由多个相同的层堆叠而成。每层包含一个**带掩码的多头自注意力层 (Masked Multi-Head Self-Attention)**（确保预测当前词时只能关注之前的词），一个**编码器-解码器注意力层 (Encoder-Decoder Attention)**（关注编码器的输出），以及一个前馈网络。
*   **位置编码 (Positional Encoding)**：由于 Transformer 没有循环结构来处理序列顺序，因此需要引入位置编码来为模型提供词语的顺序信息。

### Python 代码示例：Transformer 核心概念 (自注意力)

这里我们只演示 Transformer 最核心的自注意力机制。

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class SelfAttention(nn.Module):
    def __init__(self, embed_dim, head_dim):
        super().__init__()
        self.embed_dim = embed_dim
        self.head_dim = head_dim
        self.sqrt_head_dim = math.sqrt(head_dim)

        self.query_proj = nn.Linear(embed_dim, head_dim)
        self.key_proj = nn.Linear(embed_dim, head_dim)
        self.value_proj = nn.Linear(embed_dim, head_dim)

    def forward(self, x, mask=None):
        # x: (batch_size, seq_len, embed_dim)
        
        # 投影到Q, K, V
        Q = self.query_proj(x) # (batch_size, seq_len, head_dim)
        K = self.key_proj(x)   # (batch_size, seq_len, head_dim)
        V = self.value_proj(x)  # (batch_size, seq_len, head_dim)

        # 计算注意力分数
        # Q * K^T -> (batch_size, seq_len, head_dim) * (batch_size, head_dim, seq_len)
        attention_scores = torch.bmm(Q, K.transpose(1, 2)) / self.sqrt_head_dim 
        # attention_scores: (batch_size, seq_len, seq_len)

        # 应用掩码 (如果需要，例如在解码器中防止看到未来信息)
        if mask is not None:
            attention_scores = attention_scores.masked_fill(mask == 0, -1e9) # 填充一个非常小的数

        # softmax得到注意力权重
        attention_weights = F.softmax(attention_scores, dim=-1)
        # attention_weights: (batch_size, seq_len, seq_len)

        # 加权求和V得到输出
        output = torch.bmm(attention_weights, V)
        # output: (batch_size, seq_len, head_dim)
        
        return output, attention_weights

# 模拟输入嵌入
batch_size = 2
seq_len = 5
embed_dim = 128
head_dim = 64 # 每个头的维度

# 随机生成一些输入嵌入
input_embeddings = torch.randn(batch_size, seq_len, embed_dim)

# 实例化自注意力层
self_attention_layer = SelfAttention(embed_dim, head_dim)

# 前向传播
output, attention_weights = self_attention_layer(input_embeddings)

print("--- 自注意力机制示例 ---")
print(f"输入嵌入形状: {input_embeddings.shape}")
print(f"自注意力输出形状: {output.shape}")
print(f"注意力权重形状 (显示每个词对其他词的关注程度): {attention_weights.shape}")
print("\n示例: 第一个批次中，第一个词对所有词的关注权重:")
print(attention_weights[0, 0, :])

# 带有掩码的自注意力（例如，模拟解码器）
# 假设我们要生成第3个词，它只能看到第0, 1, 2个词
# 生成一个下三角矩阵作为mask
def generate_causal_mask(seq_len):
    mask = torch.ones(seq_len, seq_len).tril() # lower triangle
    return mask.unsqueeze(0).unsqueeze(0) # (1, 1, seq_len, seq_len) to be broadcastable

mask_example = generate_causal_mask(seq_len)
print(f"\n因果掩码形状: {mask_example.shape}")
# 注意：nn.MultiheadAttention 内部处理 head_dim 到 embed_dim 的转换，这里只是简化展示单个头的自注意力
# 对于MultiHead Attention，输出会被concatenate再通过一个线性层投影回embed_dim
output_masked, attention_weights_masked = self_attention_layer(input_embeddings, mask=mask_example.squeeze(0))
print("\n示例: 带有因果掩码的第一个批次中，第一个词对所有词的关注权重:")
print(attention_weights_masked[0, 0, :]) # 注意最后一个词的注意力权重，会被掩码为0或非常小的值


--- 自注意力机制示例 ---
输入嵌入形状: torch.Size([2, 5, 128])
自注意力输出形状: torch.Size([2, 5, 64])
注意力权重形状 (显示每个词对其他词的关注程度): torch.Size([2, 5, 5])

示例: 第一个批次中，第一个词对所有词的关注权重:
tensor([0.2274, 0.2366, 0.1803, 0.1939, 0.1617], grad_fn=<SliceBackward0>)

因果掩码形状: torch.Size([1, 1, 5, 5])

示例: 带有因果掩码的第一个批次中，第一个词对所有词的关注权重:
tensor([1., 0., 0., 0., 0.], grad_fn=<SliceBackward0>)
